# Figure Manipulation

After creating a figure with `xpx()`, you can manipulate it using standard Plotly methods.
This notebook shows what works out of the box, and where `update_traces` from xarray-plotly helps.

In [ ]:
import numpy as np
import plotly.express as px
import xarray as xr

from xarray_plotly import config, update_traces, xpx

config.notebook()

In [ ]:
# 4D DataArray: scenario x metric x year x country
df_gap = px.data.gapminder()
countries = ["United States", "China", "Germany", "Brazil"]
metrics = ["lifeExp", "gdpPercap"]

# Build base 3D array (metric x year x country)
arrays = []
for metric in metrics:
    df_pivot = df_gap[df_gap["country"].isin(countries)].pivot(
        index="year", columns="country", values=metric
    )
    arrays.append(df_pivot.values)

base_3d = np.stack(arrays)

# Add scenario dimension (4D): original + 10% higher
scenarios = ["baseline", "optimistic"]
data_4d = np.stack([base_3d, base_3d * 1.1])

da = xr.DataArray(
    data_4d,
    dims=["scenario", "metric", "year", "country"],
    coords={
        "scenario": scenarios,
        "metric": metrics,
        "year": df_pivot.index.tolist(),
        "country": df_pivot.columns.tolist(),
    },
    name="value",
)
da

---
## Standard Plotly Methods

All standard Plotly manipulation methods work on figures created with `xpx()`.

In [ ]:
# Simple 2D slice
fig = xpx(da.sel(scenario="baseline", metric="lifeExp")).line()
fig

In [ ]:
# Layout
fig.update_layout(title="Life Expectancy Over Time", template="plotly_white")

# All traces
fig.update_traces(line_width=3)

# Specific trace by name
fig.update_traces(line_dash="dot", selector={"name": "Germany"})

# Axes
fig.update_xaxes(title="Year", showgrid=False)
fig.update_yaxes(title="Life Expectancy (years)", range=[40, 85])

# Reference line
fig.add_hline(y=70, line_dash="dash", line_color="gray", annotation_text="Target")

fig

---
## Faceted Plots

`update_traces`, `update_xaxes`, `update_yaxes` work across all facets.

In [ ]:
# Facet by metric, color by country
fig = xpx(da.sel(scenario="baseline")).line(facet_col="metric")
fig

In [ ]:
# Update ALL traces across all facets
fig.update_traces(line_width=2)

# Update ALL axes
fig.update_xaxes(showgrid=False)

# Target specific facet (1-indexed)
fig.update_yaxes(type="log", col=2)  # log scale only for gdpPercap

fig

### Grid layout with facet_row

In [ ]:
# 2x2 grid: scenario x metric
fig = xpx(da).line(facet_col="metric", facet_row="scenario")

fig.update_traces(line_width=2)
fig.update_yaxes(type="log", col=2)  # log scale for gdpPercap column
fig

---
## Animation: The Pain Point

Plotly's `fig.update_traces()` does **not** update animation frames. This is the main gotcha.

In [ ]:
# Animated bar chart
fig = xpx(da.sel(scenario="baseline", metric="gdpPercap")).bar(animation_frame="year")
fig

In [ ]:
# This only affects the INITIAL view!
fig.update_traces(marker_color="red")

print(f"Base trace color: {fig.data[0].marker.color}")
print(f"Frame 0 trace color: {fig.frames[0].data[0].marker.color}")

In [ ]:
# Play the animation - it reverts to original colors
fig

### Solution: `update_traces` from xarray-plotly

This helper updates both base traces and all animation frames.

In [ ]:
fig = xpx(da.sel(scenario="baseline", metric="gdpPercap")).bar(animation_frame="year")

update_traces(fig, marker_color="red", marker_opacity=0.8)

print(f"Base trace color: {fig.data[0].marker.color}")
print(f"Frame 0 trace color: {fig.frames[0].data[0].marker.color}")

In [ ]:
# Now the style persists through animation
fig

### Selective updates with selector

In [ ]:
fig = xpx(da.sel(scenario="baseline", metric="lifeExp")).line(x="year")

# Highlight specific countries
update_traces(fig, selector={"name": "China"}, line_color="red", line_width=4)
update_traces(fig, selector={"name": "United States"}, line_color="blue", line_width=4)

fig

### Unified hover with animation

A common pattern: unified hover mode with custom formatting.

- **Layout** (`hovermode`, spikes): Standard Plotly works fine
- **Traces** (`hovertemplate`): Use `update_traces()` for animation support

In [ ]:
fig = xpx(da.sel(metric="gdpPercap")).line(x="year", animation_frame="scenario")

# Layout settings - standard Plotly
fig.update_layout(hovermode="x unified")
fig.update_xaxes(showspikes=True, spikecolor="gray", spikethickness=1)

# Trace settings - use update_traces for animation support
update_traces(fig, hovertemplate="<b>%{fullData.name}</b>: $%{y:,.0f}<extra></extra>")

fig

### Facets + Animation

In [ ]:
# Facet by metric, animate by scenario
fig = xpx(da).line(facet_col="metric", animation_frame="scenario")

# Standard Plotly for layout
fig.update_yaxes(type="log", col=2)

# update_traces for trace properties with animation
update_traces(fig, line_width=3)
update_traces(fig, selector={"name": "China"}, line_dash="dot")

fig

---
## Summary

| Method | Static/Faceted | Animated |
|--------|----------------|----------|
| `fig.update_layout()` | ✅ | ✅ |
| `fig.update_xaxes()` / `fig.update_yaxes()` | ✅ | ✅ |
| `fig.add_hline()` / `fig.add_vline()` | ✅ | ✅ |
| `fig.update_traces()` | ✅ | ❌ base only |
| `update_traces(fig, ...)` | ✅ | ✅ all frames |